In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

print("Pandas versión es :", pd.__version__)

Pandas versión es : 2.0.3


In [7]:
# Nombres de columnas (UNSW-NB15_1)
columns = [
    'srcip','sport','dstip','dsport','proto','state','dur','sbytes','dbytes',
    'sttl','dttl','sloss','dloss','service','Sload','Dload','Spkts','Dpkts',
    'swin','dwin','stcpb','dtcpb','smeansz','dmeansz','trans_depth','res_bdy_len',
    'Sjit','Djit','Stime','Ltime','Sintpkt','Dintpkt','tcprtt','synack','ackdat',
    'is_sm_ips_ports','ct_state_ttl','ct_flw_http_mthd','is_ftp_login','ct_ftp_cmd',
    'ct_srv_src','ct_srv_dst','ct_dst_ltm','ct_src_ltm','ct_src_dport_ltm',
    'ct_dst_sport_ltm','ct_dst_src_ltm','attack_cat','Label'
]

# Cargar CSV
df = pd.read_csv('UNSW-NB15_1.csv', names=columns, header=None, low_memory=False)

# Revisar las primeras 5 filas
df.head()


,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,0,3,7,1,3,1,1,1,NaN,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,NaN,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,NaN,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,NaN,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,NaN,0


In [15]:
# Convertir los Nan de  columna attack_cat a normal 
df['attack_cat'] = df['attack_cat'].fillna('Normal')
# Comprobar todos los tipos de datos de la columna attack_cat
print(df['attack_cat'].unique())
# La cantidad de registros de esa columna
print(df['attack_cat'].value_counts())


['Normal' 'Exploits' 'Reconnaissance' 'DoS' 'Generic' 'Shellcode'
 ' Fuzzers' 'Worms' 'Backdoors' 'Analysis']
attack_cat
Normal            677786
Generic             7522
Exploits            5409
 Fuzzers            5051
Reconnaissance      1759
DoS                 1167
Backdoors            534
Analysis             526
Shellcode            223
Worms                 24
Name: count, dtype: int64


In [21]:
# Creación de columna attack_cat_num a consecuencia de ser una columna categorica
df['attack_cat_num'] = df['attack_cat'].map({
    'Normal': 0,
    'Generic': 1,
    'Exploits': 2,
    ' Fuzzers': 3,       
    'Reconnaissance': 4,
    'DoS': 5,
    'Backdoors': 6,
    'Analysis': 7,
    'Shellcode': 8,
    'Worms': 9
})

# Revisar que se haya creado correctamente
print(df[['attack_cat', 'attack_cat_num']].head())
print(df['attack_cat_num'].value_counts())


  attack_cat  attack_cat_num
0     Normal               0
1     Normal               0
2     Normal               0
3     Normal               0
4     Normal               0
attack_cat_num
0    677786
1      7522
2      5409
3      5051
4      1759
5      1167
6       534
7       526
8       223
9        24
Name: count, dtype: int64


In [24]:
# Columnas a revisar
cat_cols = ['proto', 'state', 'service']

for col in cat_cols:
    print(f"Columna: {col}")
    print("Valores únicos:", df[col].unique())
    print("Conteo de registros:\n", df[col].value_counts())
    print("-" * 50)


Columna: proto
Valores únicos: ['udp' 'arp' 'tcp' 'ospf' 'icmp' 'igmp' 'sctp' 'udt' 'sep' 'sun-nd'
 'swipe' 'mobile' 'pim' 'rtp' 'ipnip' 'ip' 'ggp' 'st2' 'egp' 'cbt' 'emcon'
 'nvp' 'igp' 'xnet' 'argus' 'bbn-rcc' 'chaos' 'pup' 'hmp' 'mux' 'dcn'
 'prm' 'trunk-1' 'xns-idp' 'trunk-2' 'leaf-1' 'leaf-2' 'irtp' 'rdp'
 'iso-tp4' 'netblt' 'mfe-nsp' 'merit-inp' '3pc' 'xtp' 'idpr' 'tp++' 'ddp'
 'idpr-cmtp' 'ipv6' 'il' 'idrp' 'ipv6-frag' 'sdrp' 'ipv6-route' 'gre'
 'rsvp' 'mhrp' 'bna' 'esp' 'i-nlsp' 'narp' 'ipv6-no' 'tlsp' 'skip'
 'ipv6-opts' 'any' 'cftp' 'sat-expak' 'kryptolan' 'rvd' 'ippc' 'sat-mon'
 'ipcv' 'visa' 'cpnx' 'cphb' 'wsn' 'pvp' 'br-sat-mon' 'wb-mon' 'wb-expak'
 'iso-ip' 'secure-vmtp' 'vmtp' 'vines' 'ttp' 'nsfnet-igp' 'dgp' 'tcf'
 'eigrp' 'sprite-rpc' 'larp' 'mtp' 'ax.25' 'ipip' 'micp' 'aes-sp3-d'
 'encap' 'etherip' 'pri-enc' 'gmtp' 'pnni' 'ifmp' 'aris' 'qnx' 'a/n'
 'scps' 'snp' 'ipcomp' 'compaq-peer' 'ipx-n-ip' 'vrrp' 'zero' 'pgm' 'iatp'
 'ddx' 'l2tp' 'srp' 'stp' 'smp' 'uti' 'sm' 'ptp

In [25]:
# Limpieza de la columna : service/state , - 430656 datos sin servicio , 2 columnas conn el state no , remplazare por el valor
# Unkown
df['state'] = df['state'].replace('no', 'Unknown')
df['service'] = df['service'].replace('-', 'Unknown')



In [27]:
# Comprobar todos los tipos de datos de la columna attack_cat
print(df['state'].unique())
# La cantidad de registros de esa columna
print(df['state'].value_counts())

['CON' 'INT' 'FIN' 'URH' 'REQ' 'ECO' 'RST' 'CLO' 'TXD' 'URN' 'Unknown'
 'ACC' 'PAR' 'MAS' 'TST' 'ECR']
state
FIN        487911
CON        187505
INT         21799
REQ          2429
CLO           111
URH           108
RST            74
ECO            26
ACC            22
PAR             4
TXD             2
URN             2
Unknown         2
MAS             2
TST             2
ECR             2
Name: count, dtype: int64


In [28]:
# Comprobar todos los tipos de datos de la columna attack_cat
print(df['service'].unique())
# La cantidad de registros de esa columna
print(df['service'].value_counts())

df.head()

['dns' 'Unknown' 'http' 'smtp' 'ftp-data' 'ftp' 'ssh' 'pop3' 'snmp' 'ssl'
 'irc' 'radius' 'dhcp']
service
Unknown     430656
dns         121170
http         55858
ftp-data     37305
smtp         23588
ftp          16531
ssh          14636
pop3           206
ssl             20
snmp            14
radius           7
dhcp             7
irc              3
Name: count, dtype: int64


,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label,attack_cat_num
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,3,7,1,3,1,1,1,Normal,0,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,2,4,2,3,1,1,2,Normal,0,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,12,8,1,2,2,1,1,Normal,0,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,6,9,1,1,1,1,1,Normal,0,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,7,9,1,1,1,1,1,Normal,0,0


In [ ]:
# Columnas categóricas que queremos convertir
cat_cols = ['proto', 'state', 'service']

# Crear variables dummy
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)  # drop_first=True elimina la primera categoría de cada columna



In [36]:
# Seleccionar todas las columnas dummy(true/false)
dummy_cols = df.select_dtypes(include='bool').columns

# Convertir True/False a 1/0
df[dummy_cols] = df[dummy_cols].astype(int)
df.head(10)


,srcip,sport,dstip,dsport,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label,attack_cat_num,proto_a/n,proto_aes-sp3-d,proto_any,proto_argus,proto_aris,proto_arp,proto_ax.25,proto_bbn-rcc,proto_bna,proto_br-sat-mon,proto_cbt,proto_cftp,proto_chaos,proto_compaq-peer,proto_cphb,proto_cpnx,proto_crtp,proto_crudp,proto_dcn,proto_ddp,proto_ddx,proto_dgp,proto_egp,proto_eigrp,proto_emcon,proto_encap,proto_esp,proto_etherip,proto_fc,proto_fire,proto_ggp,proto_gmtp,proto_gre,proto_hmp,proto_i-nlsp,proto_iatp,proto_ib,proto_icmp,proto_idpr,proto_idpr-cmtp,proto_idrp,proto_ifmp,proto_igmp,proto_igp,proto_il,proto_ip,proto_ipcomp,proto_ipcv,proto_ipip,proto_iplt,proto_ipnip,proto_ippc,proto_ipv6,proto_ipv6-frag,proto_ipv6-no,proto_ipv6-opts,proto_ipv6-route,proto_ipx-n-ip,proto_irtp,proto_isis,proto_iso-ip,proto_iso-tp4,proto_kryptolan,proto_l2tp,proto_larp,proto_leaf-1,proto_leaf-2,proto_merit-inp,proto_mfe-nsp,proto_mhrp,proto_micp,proto_mobile,proto_mtp,proto_mux,proto_narp,proto_netblt,proto_nsfnet-igp,proto_nvp,proto_ospf,proto_pgm,proto_pim,proto_pipe,proto_pnni,proto_pri-enc,proto_prm,proto_ptp,proto_pup,proto_pvp,proto_qnx,proto_rdp,proto_rsvp,proto_rtp,proto_rvd,proto_sat-expak,proto_sat-mon,proto_sccopmce,proto_scps,proto_sctp,proto_sdrp,proto_secure-vmtp,proto_sep,proto_skip,proto_sm,proto_smp,proto_snp,proto_sprite-rpc,proto_sps,proto_srp,proto_st2,proto_stp,proto_sun-nd,proto_swipe,proto_tcf,proto_tcp,proto_tlsp,proto_tp++,proto_trunk-1,proto_trunk-2,proto_ttp,proto_udp,proto_udt,proto_unas,proto_uti,proto_vines,proto_visa,proto_vmtp,proto_vrrp,proto_wb-expak,proto_wb-mon,proto_wsn,proto_xnet,proto_xns-idp,proto_xtp,proto_zero,state_CLO,state_CON,state_ECO,state_ECR,state_FIN,state_INT,state_MAS,state_PAR,state_REQ,state_RST,state_TST,state_TXD,state_URH,state_URN,state_Unknown,service_dhcp,service_dns,service_ftp,service_ftp-data,service_http,service_irc,service_pop3,service_radius,service_smtp,service_snmp,service_ssh,service_ssl
0,59.166.0.0,1390,149.171.126.6,53,0.001055,132,164,31,29,0,0,500473.93750,621800.93750,2,2,0,0,0,0,66,82,0,0,0.00000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0,0,0,3,7,1,3,1,1,1,Normal,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,59.166.0.0,33661,149.171.126.9,1024,0.036133,528,304,31,29,0,0,87676.08594,50480.17188,4,4,0,0,0,0,132,76,0,0,9.89101,10.682733,1421927414,1421927414,7.005,7.564333,0.0,0.0,0.0,0,0,0,0,0,2,4,2,3,1,1,2,Normal,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,59.166.0.6,1464,149.171.126.7,53,0.001119,146,178,31,29,0,0,521894.53130,636282.37500,2,2,0,0,0,0,73,89,0,0,0.00000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0,0,0,12,8,1,2,2,1,1,Normal,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,59.166.0.5,3593,149.171.126.5,53,0.001209,132,164,31,29,0,0,436724.56250,542597.18750,2,2,0,0,0,0,66,82,0,0,0.0000

In [38]:
# Para guardar todo el progreso en un csv
df.to_csv('UNSW_NB15_clean1.csv', index=False)
